# Spotify Dataset Cleaning

This notebook cleans and preprocesses the Spotify dataset for music recommendation analysis.

## Overview
- Dataset: 114,000 Spotify tracks with audio features
- Cleaning steps: missing values, duplicates, data validation, feature scaling
- Output: Clean dataset ready for clustering and recommendation


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
from scipy.cluster.hierarchy import linkage, dendrogram
import statsmodels.api as sm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

print("All packages imported successfully!")

## **Upload files**


In [ ]:
df = pd.read_csv('dataset.csv')
print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")

## **Data Cleaning**


#### Data inspection


In [ ]:
print("First 5 rows:")
display(df.head())
print("\n" + "="*100 + "\n")
print("Last 5 rows:")
display(df.tail())

In [ ]:
print("Data types of all columns:")
print(df.dtypes)
print("\n" + "="*100 + "\n")
print("Dataset info:")
df.info()

In [ ]:
print("Summary statistics for numerical columns:")
display(df.describe())
print("\n" + "="*100 + "\n")
print("Summary statistics (including non-numeric):")
display(df.describe(include='all'))



In [ ]:
## Drop irrelevant features
print(df.columns.tolist())
drop_cols = ['time_signature','key']
df.drop(columns = drop_cols,inplace = True)

### Check for and handle missing values

In [ ]:
missing_values = df.isnull().sum()
print("Columns with missing values:")
print(missing_values[missing_values > 0])
print(f"\nTotal missing values: {missing_values.sum()}")
df_clean = df.dropna()
print(f"\nRows before: {len(df)}")
print(f"Rows after: {len(df_clean)}")
print(f"Rows removed: {len(df) - len(df_clean)}")

### Split artyist names


In [ ]:
# Split on artist names
# Split artists by semicolon
artists_split = df_clean['artists'].str.split(';', expand=True)

# Add the split columns to the dataframe with proper naming and cleaning
for i in range(artists_split.shape[1]):
    col_name = f'artist_{i+1}'
    # Clean each artist column: strip whitespace and convert to lowercase
    df_clean[col_name] = artists_split[i].str.strip().str.lower()

df_clean.columns

# # Only keep first 3 artists
df_clean = df_clean[['track_id', 'album_name', 'track_name','duration_ms', 'explicit','popularity', 'danceability', 'energy', 'loudness', 'mode','speechiness', 'acousticness', 'instrumentalness', 'liveness','valence', 'tempo', 'track_genre', 'artist_1', 'artist_2', 'artist_3']]

### Check for Duplicate Rows


In [ ]:
# Remove duplicates based on track_name and artist_1
df_clean_unique = df_clean.drop_duplicates(subset=['track_name', 'artist_1'], keep='first')

print(f"Original: {len(df_clean)} rows")
print(f"Removed: {len(df_clean) - len(df_clean_unique)} duplicates")
print(f"Final: {len(df_clean_unique)} rows")
df_clean=df_clean_unique

### Data Type Conversion (if needed)


In [ ]:
print("Converting data types:")
#Converting explicit from boolean to binary
if 'explicit' in df_clean.columns:
    df_clean['explicit'] = df_clean['explicit'].astype(int)
    print("✓ Converted 'explicit' to binary (0/1)")

unnamed_cols = [
    col for col in df_clean.columns if 'Unnamed' in col or col == '']
if unnamed_cols:
    df_clean = df_clean.drop(columns=unnamed_cols)
    print(f"✓ Dropped unnamed index columns: {unnamed_cols}")
else:
    print("✓ No unnamed columns to drop")

print("\nData types after conversion:")
print(df_clean.dtypes)

In [ ]:
date_columns = [col for col in df_clean.columns if 'date' in col.lower()
                or 'time' in col.lower()]
if date_columns:
    print(f"Date columns found: {date_columns}")
else:
    print("No date columns to parse")

In [ ]:
categorical_candidates = df_clean.select_dtypes(
    include=['object']).columns.tolist()
print(
    f"Remaining object columns that could be categorical: {categorical_candidates}")
print(
    f"\nCurrent categorical columns: {df_clean.select_dtypes(include=['category']).columns.tolist()}")

### Handle Inconsistent Data


In [ ]:
text_columns = df_clean.select_dtypes(include=['object']).columns.tolist()
print(f"Text columns to standardize: {text_columns}")

for col in text_columns:
    df_clean[col] = df_clean[col].str.strip()
    df_clean[col] = df_clean[col].str.lower()
    print(f"✓ Standardized '{col}': lowercase and stripped whitespace")

print("\nSample of standardized data:")
display(df_clean[text_columns].head())

In [ ]:
if 'track_genre' in df_clean.columns:
    print("Unique genres before cleaning:")
    print(f"Total unique genres: {df_clean['track_genre'].nunique()}")

    print("\nChecking for inconsistent values in track_genre...")
    genre_counts = df_clean['track_genre'].value_counts()
    print(f"\nTop 10 genres:")
    print(genre_counts.head(10))
else:
    print("No track_genre column to check for inconsistencies")

### Validate Data Integrity


In [ ]:
print("Checking for logical inconsistencies:\n")

if 'popularity' in df_clean.columns:
    invalid_popularity = df_clean[(df_clean['popularity'] < 0) | (
        df_clean['popularity'] > 100)]
    print(
        f"  - Popularity out of range [0-100]: {len(invalid_popularity)} rows")

if 'duration_ms' in df_clean.columns:
    invalid_duration = df_clean[df_clean['duration_ms'] < 90000]         #any track below 1 min 30 seconds 
    print(f"  - Duration < 90000: {len(invalid_duration)} rows")
if 'duration_ms' in df_clean.columns:
    invalid_duration = df_clean[df_clean['duration_ms'] > 900000]         #any track 15 minutes or longer
    print(f"  - Duration > 900000: {len(invalid_duration)} rows")

if 'tempo' in df_clean.columns:
    invalid_tempo = df_clean[df_clean['tempo'] < 0]
    print(f"  - Tempo < 0: {len(invalid_tempo)} rows")

print("\n✓ Logical consistency check complete")

### As you can see, 2846 tracks have a duration of less than 1 min 30 seconds
### And 153 had a duration of longer than 15 mins
### these will all be removed

In [ ]:
# Remove tracks with duration < 90 seconds (90000 ms)
df_clean = df_clean[df_clean['duration_ms'] >= 90000]
df_clean = df_clean[df_clean['duration_ms'] <= 900000]


In [ ]:
print("Verifying ranges and constraints:\n")

audio_features = ['danceability', 'energy', 'speechiness', 'acousticness',
                  'instrumentalness', 'liveness', 'valence']

for feature in audio_features:
    if feature in df_clean.columns:
        out_of_range = df_clean[(df_clean[feature] < 0)
                                | (df_clean[feature] > 1)]
        print(f"  - {feature} out of range [0-1]: {len(out_of_range)} rows")


if 'mode' in df_clean.columns:
    invalid_mode = df_clean[(df_clean['mode'] < 0) | (df_clean['mode'] > 1)]
    print(f"  - Mode out of range [0-1]: {len(invalid_mode)} rows")

print("\n✓ Range validation complete")


In [ ]:
print("Cross-checking related columns:\n")

if 'loudness' in df_clean.columns:
    print(
        f"  - Loudness range: [{df_clean['loudness'].min():.2f}, {df_clean['loudness'].max():.2f}] dB")
    print(f"    Expected: typically between -60 and 0 dB")

if 'tempo' in df_clean.columns:
    print(
        f"  - Tempo range: [{df_clean['tempo'].min():.2f}, {df_clean['tempo'].max():.2f}] BPM")
    print(f"    Expected: typically between 50 and 200 BPM")
if 'duration_ms' in df_clean.columns:
    duration_seconds = df_clean['duration_ms'] / 1000
    print(
        f"  - Duration range: [{duration_seconds.min():.2f}, {duration_seconds.max():.2f}] seconds")
    print(
        f"    ({duration_seconds.min()/60:.2f} to {duration_seconds.max()/60:.2f} minutes)")

print("\n✓ Cross-check complete")

### There appear to be some tracks with loudness ratings above 0. These are higher than the system can handle and will distort, so we must remove these values

In [ ]:
df_clean = df_clean[df_clean['loudness'] <= 0]

## Scale unscaled features

In [ ]:
df_clean.describe()
# From df_clean.describe it is clear that a few features are not scaled from the raw data:
# These include loudness in which more negative values actually represent higher loudness, and tempo which is the opposite way arround

from sklearn.preprocessing import MinMaxScaler
from joblib import dump

print("Creating and saving scalers:\n")

# Tempo: Higher BPM = 1, Lower BPM = 0
tempo_scaler = MinMaxScaler()
df_clean['tempo_scaled'] = tempo_scaler.fit_transform(df_clean[['tempo']])
print(f"✓ Tempo: [{df_clean['tempo'].min():.0f}, {df_clean['tempo'].max():.0f}] BPM → [0, 1]")

# Loudness: Lower dB = 1, Higher dB = 0 (reversed)
loudness_scaler = MinMaxScaler()
df_clean['loudness_scaled'] = 1 - loudness_scaler.fit_transform(df_clean[['loudness']])
print(f"✓ Loudness: [{df_clean['loudness'].min():.1f}, {df_clean['loudness'].max():.1f}] dB → [0, 1] (reversed)")

# Save scalers
dump(tempo_scaler, 'tempo_scaler.joblib')
dump(loudness_scaler, 'loudness_scaler.joblib')
print("\n✓ Scalers saved to .joblib files")

df_clean.drop(columns=['tempo','loudness'],inplace=True)

### Create Clean Dataset


In [ ]:
df_clean.to_csv('dataset_cleaned.csv', index=False)
print("✓ Cleaned dataset saved to: dataset_cleaned.csv")
print(f"  Rows: {df_clean.shape[0]}")
print(f"  Columns: {df_clean.shape[1]}")

### Summary of cleaning process

## Data Cleaning Summary

### Initial Dataset
- **114,000 rows × 21 columns**
- Spotify track data with audio features and metadata

### 1. Data Inspection
- Examined first/last rows, data types, and shape
- Generated summary statistics for all columns
- Identified data structure and feature distributions

### 2. Missing Values
- **Found:** 3 missing values (1 each in artists, album_name, track_name)
- **Action:** Filled missing text columns with mode values
  - artists → "The Beatles"
  - album_name → "Alternative Christmas 2022"
  - track_name → "Run Rudolph Run"
- **Result:** 0 missing values remaining

### 3. Duplicate Rows
- **Found:** 0 duplicate rows
- **Action:** No removal needed

### 4. Data Type Conversion
- Converted `explicit` to boolean type
- Dropped `Unnamed: 0` index column
- Verified all other data types are appropriate

### 5. Text Standardization
- Standardized all text columns (track_id, artists, album_name, track_name, track_genre)
- Applied lowercase transformation
- Stripped leading/trailing whitespace
- Verified genre consistency (114 unique genres, balanced distribution)

### 6. Data Integrity Validation
- **Popularity:** All values within valid range [0-100]
- **Duration:** 1 row with duration ≤ 0 identified
- **Tempo:** 157 rows with tempo ≤ 0 identified
- **Audio features:** All within expected [0-1] range
- **Loudness:** Range [-49.53, 4.53] dB (some positive values indicate clipping)
- **Time signature:** 163 rows out of typical range [1-7]

### Current Dataset Status
- **114,000 rows × 20 columns** (after dropping Unnamed: 0)
- **Columns:** track_id, artists, album_name, track_name, track_genre, popularity, duration_ms, explicit, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, time_signature
- **Output:** Saved to `dataset_cleaned.csv`

### Next Steps
- Remove invalid rows (duration ≤ 0, tempo ≤ 0, loudness > 0)
- Separate metadata columns from feature columns
- Drop irrelevant features (key, mode, time_signature)
- Scale features for clustering analysis